<a href="https://colab.research.google.com/github/Devikarb/CyberIntelExtractor/blob/main/CyberIntel%26Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-whois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import requests
import whois
import socket
import json

# Replace with your actual API keys
ABUSEIPDB_API_KEY = 'API key'
URLSCAN_API_KEY = 'API key'
VIRUSTOTAL_API_KEY = 'API key'
ALIENVAULT_API_KEY = 'API key'
MALSHARE_API_KEY = 'API key'

# Domain-related functions
def get_ip_from_domain(domain):
    try:
        ip_address = socket.gethostbyname(domain)
        return ip_address
    except socket.error as err:
        print(f"Error resolving IP for {domain}: {err}")
        return None

def get_abuseipdb_info(ip_address):
    if not ip_address:
        return {"error": "Unable to resolve IP address"}

    url = f'https://api.abuseipdb.com/api/v2/check'
    params = {'ipAddress': ip_address, 'maxAgeInDays': 90}
    headers = {'Accept': 'application/json', 'Key': ABUSEIPDB_API_KEY}
    response = requests.get(url, headers=headers, params=params)
    return response.json()

def get_whois_info(domain_or_ip):
    try:
        w = whois.whois(domain_or_ip)
        return w
    except Exception as e:
        return {"error": str(e)}

def get_urlscan_info(domain):
    url = 'https://urlscan.io/api/v1/search/'
    headers = {'API-Key': URLSCAN_API_KEY}
    params = {'q': domain}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if 'results' in data and data['results']:
        screenshot_url = data['results'][0].get('screenshot', None)
        if screenshot_url:
            return {
                'screenshot_url': screenshot_url,
                'results': data['results']
            }
        else:
            return {'error': 'Screenshot not available in results'}
    else:
        return {'error': 'No results found or unexpected response format'}

def get_virustotal_info_domain(domain):
    url = f'https://www.virustotal.com/api/v3/domains/{domain}'
    headers = {
        'x-apikey': VIRUSTOTAL_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()

def get_virustotal_info_ip(ip):
    url = f'https://www.virustotal.com/api/v3/ip_addresses/{ip}'
    headers = {
        'x-apikey': VIRUSTOTAL_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()

def get_alienvault_info_domain(domain):
    url = f'https://otx.alienvault.com/api/v1/indicators/domain/{domain}/general'
    headers = {
        'X-OTX-API-KEY': ALIENVAULT_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()

# Hash-related functions
def get_virustotal_info_hash(hash_value):
    url = f'https://www.virustotal.com/api/v3/files/{hash_value}'
    headers = {
        'x-apikey': VIRUSTOTAL_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()

def get_alienvault_info_hash(hash_value):
    url = f'https://otx.alienvault.com/api/v1/indicators/file/{hash_value}/general'
    headers = {
        'X-OTX-API-KEY': ALIENVAULT_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()

def get_malshare_info(hash_value):
    url = f'https://malshare.com/api.php?api_key={MALSHARE_API_KEY}&action=search&query={hash_value}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # MalShare returns a list of dictionaries with file details
    else:
        return {"error": f"Error occurred: {response.status_code}"}

# Formatting functions
def format_abuseipdb_info(info):
    if 'data' in info:
        data = info['data']
        detection = "Malicious" if data.get('totalReports', 0) > 0 else "Not Malicious"
        return (
            f"ABUSE IP (Detection: {detection}):\n\n"
            f"This IP was reported {data.get('totalReports', 'N/A')} times.\n"
            f"Confidence of Abuse: {data.get('abuseConfidenceScore', 'N/A')}%\n"
            f"ISP: {data.get('isp', 'N/A')}\n"
            f"Usage Type: {data.get('usageType', 'N/A')}\n"
            f"Hostname(s): {', '.join(data.get('hostnames', []))}\n"
            f"Domain Name: {data.get('domain', 'N/A')}\n"
            f"Country: {data.get('countryCode', 'N/A')}\n"
            f"City: {data.get('city', 'N/A')}"
        )
    else:
        return "ABUSE IP: No data available"

def format_whois_info(info):
    if 'error' in info:
        return f"WHOIS Information:\n\n{info['error']}"

    domain_names = info.domain_name
    if isinstance(domain_names, list):
        domain_names_str = ', '.join(domain_names)
    elif isinstance(domain_names, str):
        domain_names_str = domain_names
    else:
        domain_names_str = 'N/A'

    return (
        f"WHOIS Information:\n\n"
        f"Domain: {domain_names_str}\n"
        f"Registrar: {info.registrar if info.registrar else 'N/A'}\n"
        f"Registered On: {info.creation_date if info.creation_date else 'N/A'}\n"
        f"Expires On: {info.expiration_date if info.expiration_date else 'N/A'}\n"
        f"Updated On: {info.updated_date if info.updated_date else 'N/A'}"
    )

def format_virustotal_info(info):
    if 'data' in info:
        data = info['data']
        attributes = data['attributes']
        return (
            f"VirusTotal:\n\n"
            f"Harmless Votes: {attributes['last_analysis_stats']['harmless']}\n"
            f"Malicious Votes: {attributes['last_analysis_stats']['malicious']}\n"
            f"Suspicious Votes: {attributes['last_analysis_stats']['suspicious']}\n"
            f"Undetected Votes: {attributes['last_analysis_stats']['undetected']}\n"
            f"Categories: {', '.join(attributes.get('categories', []))}\n"
            f"Reputation: {attributes.get('reputation', 'N/A')}"
        )
    else:
        return "VirusTotal: No data available"

def format_alienvault_info(info):
    if 'pulse_info' in info:
        pulses = info['pulse_info'].get('pulses', [])
        if not pulses:
            return "AlienVault OTX:\n\nNo pulses available"

        most_recent_pulse = pulses[0]
        recent_description = most_recent_pulse.get('description', 'No description available')
        tags = most_recent_pulse.get('tags', [])

        top_tags = ', '.join(tags[:3])

        return (
            f"AlienVault OTX:\n\n"
            f"Pulse Count: {len(pulses)}\n"
            f"Most Recent Pulse Description: {recent_description}\n"
            f"Top Tags: {top_tags if tags else 'No tags available'}"
        )
    else:
        return "AlienVault OTX:\n\nNo data available"

def format_urlscan_info(info):
    if 'error' in info:
        return f"URLScan:\n\n{info['error']}"
    return "URLScan:\n\nNo results found or unexpected response format"

def format_malshare_info(info):
    if isinstance(info, list) and len(info) > 0:
        file_info = info[0]  # MalShare may return multiple entries, taking the first one for simplicity
        return (
            f"MalShare:\n"
            f"SHA1: {file_info.get('sha1', 'N/A')}\n"
            f"SHA256: {file_info.get('sha256', 'N/A')}\n"
            f"MD5: {file_info.get('md5', 'N/A')}\n"
            f"File Type: {file_info.get('F_TYPE', 'N/A')}\n"
            f"First Seen: {file_info.get('first_seen', 'N/A')}"
        )
    else:
        return "MalShare: No data available or error occurred"


def format_results_domain(index, domain, abuseipdb_info, whois_info, urlscan_info, virustotal_info, alienvault_info):
    print(f"{index}. Domain: {domain}\n")
    print("- "+format_abuseipdb_info(abuseipdb_info)+"\n")
    print("- " + format_whois_info(whois_info)+"\n")
    print("- " + format_virustotal_info(virustotal_info)+"\n")
    print("- " + format_alienvault_info(alienvault_info)+"\n")
    print("- " + format_urlscan_info(urlscan_info)+"\n")
    print("_" * 60 +"\n")

def format_results_ip(index, ip, abuseipdb_info, virustotal_info, whois_info):
    print(f"{index}. IP: {ip}\n")
    print("- " +format_abuseipdb_info(abuseipdb_info)+"\n")
    print("- " + format_virustotal_info(virustotal_info) +"\n")
    print("- " + format_whois_info(whois_info)+"\n")
    print("_" * 60 +"\n")

def format_results_hash(index, hash_value, virustotal_info, alienvault_info, malshare_info):
    print(f"{index}. Hash: {hash_value}\n")
    print("- " +format_virustotal_info(virustotal_info)+"\n")
    print("- " + format_alienvault_info(alienvault_info)+"\n")
    print("- " + format_malshare_info(malshare_info)+"\n")
    print("_" * 60 +"\n")

# The generate_content function to extract hashes, IPs, and domains from the text

def generate_content(api_key, text):
    # Construct the prompt asking for a structured response with hashes, IPs, and domains
    prompt = (
        "Process the following text and extract the following items into separate lists: "
        "1) Hashes (MD5, SHA1, SHA256, etc.), 2) IP addresses, and 3) Domains. "
        "Return the results in a JSON object with the keys 'hashes', 'ips', and 'domains', "
        "each containing an array of extracted values. Here is the text:\n\n"
        f"{text}"
    )

    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ],
        "generationConfig": {
            "temperature": 0.7,
            "topP": 0.9,
            "topK": 50,
            "maxOutputTokens": 2048
        }
    }

    # API endpoint
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_key}"

    # Send the POST request
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers, json=payload)

    # Handle the response
    if response.status_code == 200:
        response_data = response.json()
        try:
            # Extract the relevant content from the response
            extracted_text = response_data["candidates"][0]["content"]["parts"][0]["text"]

            # Remove any code block markers or extraneous characters
            cleaned_text = extracted_text.strip('```json').strip()

            # Remove any remaining backticks and unnecessary whitespace
            cleaned_text = cleaned_text.replace('```', '').strip()

            # Parse the cleaned text as JSON
            structured_data = json.loads(cleaned_text)
            hashes = structured_data.get("hashes", [])
            ips = structured_data.get("ips", [])
            domains = structured_data.get("domains", [])

            return {
                "hashes": hashes,
                "ips": ips,
                "domains": domains
            }
        except (KeyError, json.JSONDecodeError) as e:
            print(f"Error: Unable to parse the JSON text from response. Details: {str(e)}")
            return None
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        return None


def main():
    while True:
        choice = input("Do you want to check a domain, IP, hash, or generate? Type 'domain', 'ip', 'hash', or 'generate' (or 'exit' to quit): ").lower()
        if choice == 'exit':
            break

        if choice == 'domain':
            domains_input = input("Enter domains separated by commas (or type 'exit' to quit): ")
            if domains_input.lower() == 'exit':
                break

            domains = [domain.strip().strip("'") for domain in domains_input.split(',')]
            processed_domains = set()

            for count, domain in enumerate(domains, start=1):
                if domain in processed_domains:
                    print(f"\n{count}. Domain: {domain} (Already processed)")
                    continue

                processed_domains.add(domain)

                abuseipdb_info = get_abuseipdb_info(get_ip_from_domain(domain))
                whois_info = get_whois_info(domain)
                urlscan_info = get_urlscan_info(domain)
                virustotal_info = get_virustotal_info_domain(domain)
                alienvault_info = get_alienvault_info_domain(domain)

                format_results_domain(count, domain, abuseipdb_info, whois_info, urlscan_info, virustotal_info, alienvault_info)

        elif choice == 'ip':
            ips_input = input("Enter IPs separated by commas (or type 'exit' to quit): ")
            if ips_input.lower() == 'exit':
                break

            ips = [ip.strip().strip("'") for ip in ips_input.split(',')]
            processed_ips = set()

            for count, ip in enumerate(ips, start=1):
                if ip in processed_ips:
                    print(f"\n{count}. IP: {ip} (Already processed)")
                    continue

                processed_ips.add(ip)

                abuseipdb_info = get_abuseipdb_info(ip)
                virustotal_info = get_virustotal_info_ip(ip)
                whois_info = get_whois_info(ip)

                format_results_ip(count, ip, abuseipdb_info, virustotal_info, whois_info)

        elif choice == 'hash':
            hashes_input = input("Enter hashes separated by commas (or type 'exit' to quit): ")
            if hashes_input.lower() == 'exit':
                break

            hashes = [hash_value.strip().strip("'") for hash_value in hashes_input.split(',')]
            processed_hashes = set()

            for count, hash_value in enumerate(hashes, start=1):
                if hash_value in processed_hashes:
                    print(f"\n{count}. Hash: {hash_value} (Already processed)")
                    continue

                processed_hashes.add(hash_value)

                virustotal_info = get_virustotal_info_hash(hash_value)
                alienvault_info = get_alienvault_info_hash(hash_value)
                malshare_info = get_malshare_info(hash_value)

                format_results_hash(count, hash_value, virustotal_info, alienvault_info, malshare_info)

        elif choice == 'generate':
            text = input("Enter the text for extraction (or type 'exit' to quit): ")
            if text.lower() == 'exit':
                break
            api_key = "Replace with Gemini API key from https://www.googleadservices.com/pagead/aclk?sa=L&ai=DChcSEwiki5GS_peIAxUZqWYCHe6yCgYYABAAGgJzbQ&co=1&ase=2&gclid=CjwKCAjwlbu2BhA3EiwA3yXyu4njh0PsndjkL6R-ZKuQRgoewxFIOPQGHpMkhjtYLQUYBrpvJPpZ0hoCqz4QAvD_BwE&ohost=www.google.com&cid=CAESVuD2R-_-zcgF-2ZzyFIz8BH77wHDHQbpD7PdhW--LPR_pcP5tyLkrvrh8p1dd5vfVg0CcSbfGoEm-dQixoYXfFXC8ppB9Pz0W3Dxv8U6rb7dPWpy-bRP&sig=AOD64_3GR2ZHKo94vR8KIQujsK98VW_jXg&q&nis=4&adurl&ved=2ahUKEwj5nIyS_peIAxUXd2wGHT8eLmwQ0Qx6BAgIEAE"
            extracted_data = generate_content(api_key, text)

            if extracted_data:
                print("\nExtracted Data:\n")
                print(f"Hashes: {', '.join(extracted_data['hashes'])}")
                print(f"IPs: {', '.join(extracted_data['ips'])}")
                print(f"Domains: {', '.join(extracted_data['domains'])}")
                print("\nResults:\n" + "=" * 60)


                for count, hash_value in enumerate(extracted_data["hashes"], start=1):
                    if count == 1:
                      print(f"\n---- Hashes ----\n")
                    virustotal_info = get_virustotal_info_hash(hash_value)
                    alienvault_info = get_alienvault_info_hash(hash_value)
                    malshare_info = get_malshare_info(hash_value)
                    format_results_hash(count, hash_value, virustotal_info, alienvault_info, malshare_info)


                for count, ip in enumerate(extracted_data["ips"], start=1):
                    if count == 1:
                      print(f"\n ---- IPs: ---- \n ")
                    abuseipdb_info = get_abuseipdb_info(ip)
                    virustotal_info = get_virustotal_info_ip(ip)
                    whois_info = get_whois_info(ip)
                    format_results_ip(count, ip, abuseipdb_info, virustotal_info, whois_info)


                for count, domain in enumerate(extracted_data["domains"], start=1):
                    if count == 1:
                      print(f"\n ---- Domains: ---- \n ")
                    abuseipdb_info = get_abuseipdb_info(get_ip_from_domain(domain))
                    whois_info = get_whois_info(domain)
                    urlscan_info = get_urlscan_info(domain)
                    virustotal_info = get_virustotal_info_domain(domain)
                    alienvault_info = get_alienvault_info_domain(domain)
                    format_results_domain(count, domain, abuseipdb_info, whois_info, urlscan_info, virustotal_info, alienvault_info)

        else:
            print("Invalid choice. Please enter 'domain', 'ip', 'hash', or 'generate'.\n")

if __name__ == '__main__':
    main()


Do you want to check a domain, IP, hash, or generate? Type 'domain', 'ip', 'hash', or 'generate' (or 'exit' to quit): generate
Enter the text for extraction (or type 'exit' to quit): lets  give some random domain example.com   random IP 198.51.100.150 and random hash 8f14e45fceea167a5a36dedd4bea2543, 

Extracted Data:

Hashes: 8f14e45fceea167a5a36dedd4bea2543
IPs: 198.51.100.150
Domains: example.com

Results:

---- Hashes ----

1. Hash: 8f14e45fceea167a5a36dedd4bea2543

- VirusTotal:

Harmless Votes: 0
Malicious Votes: 0
Suspicious Votes: 0
Undetected Votes: 65
Categories: 
Reputation: 29

- AlienVault OTX:

No pulses available

- MalShare: No data available or error occurred

____________________________________________________________


 ---- IPs: ---- 
 
1. IP: 198.51.100.150

- ABUSE IP (Detection: Not Malicious):

This IP was reported 0 times.
Confidence of Abuse: 0%
ISP: Test Net
Usage Type: Reserved
Hostname(s): 
Domain Name: None
Country: None
City: N/A

- VirusTotal:

Harmless